# Crypto Exchange Liquidity Health Index (LHI) Demo

This notebook demonstrates our internal liquidity monitoring system for crypto markets.

**Purpose**: Help trade ops team identify when market conditions get sketchy and execution might be problematic.

**Approach**: Practical over perfect - we want something that actually helps with trading decisions.

In [ ]:
# Import our custom analyzer
from lhi_analyzer import LiquidityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print("Liquidity Health Index (LHI) Analysis System")
print("Built for practical trade ops use")

## Step 1: Load and Examine Data

We're working with minute-level crypto market data. Nothing fancy - just the basics we need for liquidity analysis.

In [ ]:
# Initialize analyzer for BTC
btc_analyzer = LiquidityAnalyzer('BTC-USDT')
btc_analyzer.load_data('data/btc_usdt_data.csv')

# Quick look at the raw data
print("Sample of raw market data:")
print(btc_analyzer.data[['timestamp', 'bid_price', 'ask_price', 'bid_volume', 'ask_volume', 'trade_volume']].head())

print(f"\nData covers {len(btc_analyzer.data)} minutes of trading")
print(f"Price range: ${btc_analyzer.data['bid_price'].min():.0f} - ${btc_analyzer.data['ask_price'].max():.0f}")

## Step 2: Compute Liquidity Metrics

These are the core metrics any trader cares about:
- **Spread**: How much does it cost to trade immediately?
- **Depth**: How much can I trade without moving the market?
- **Volume**: Is this normal or unusual activity?
- **Volatility**: How jumpy is the price?

In [ ]:
# Compute the basic metrics
btc_analyzer.compute_basic_metrics()

# Show some key stats
metrics = btc_analyzer.metrics

print("LIQUIDITY METRICS SUMMARY")
print("=" * 30)
print(f"Average Spread: {metrics['spread_bps'].mean():.1f} basis points")
print(f"Worst Spread: {metrics['spread_bps'].max():.1f} basis points")
print(f"Average Depth: {metrics['total_depth'].mean():.1f} coins")
print(f"Thinnest Depth: {metrics['total_depth'].min():.1f} coins")
print(f"Average Volume Ratio: {metrics['volume_ratio'].mean():.1f}x normal")
print(f"Biggest Volume Spike: {metrics['volume_ratio'].max():.1f}x normal")

# Quick visualization of key metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('BTC-USDT Raw Liquidity Metrics', fontsize=14)

# Spread over time
axes[0,0].plot(metrics['timestamp'], metrics['spread_bps'], alpha=0.7, linewidth=0.5)
axes[0,0].set_title('Bid-Ask Spread (bps)')
axes[0,0].set_ylabel('Basis Points')

# Depth over time
axes[0,1].plot(metrics['timestamp'], metrics['total_depth'], alpha=0.7, linewidth=0.5)
axes[0,1].set_title('Order Book Depth')
axes[0,1].set_ylabel('Total Volume')

# Volume ratio
axes[1,0].plot(metrics['timestamp'], metrics['volume_ratio'], alpha=0.7, linewidth=0.5)
axes[1,0].axhline(y=3, color='red', linestyle='--', alpha=0.5, label='Spike Threshold')
axes[1,0].set_title('Volume Intensity')
axes[1,0].set_ylabel('Ratio vs 1h Average')
axes[1,0].legend()

# Price for context
axes[1,1].plot(metrics['timestamp'], metrics['mid_price'], alpha=0.7, linewidth=0.5, color='black')
axes[1,1].set_title('Price (for context)')
axes[1,1].set_ylabel('USDT')

plt.tight_layout()
plt.show()

## Step 3: Compute Liquidity Health Index (LHI)

This is where we combine everything into a single 0-100 score.

**Scoring Logic**:
- 80-100: Healthy liquidity (normal trading)
- 50-80: Mild stress (be a bit careful)
- 0-50: Significant stress (reduce size, use limits)

**Weights** (based on what matters for execution):
- Spread: 40% (directly affects cost)
- Depth: 30% (affects market impact)
- Volume: 15% (unusual activity is concerning)
- Volatility: 15% (affects slippage)

In [ ]:
# Compute the LHI score
btc_analyzer.compute_lhi_score()

lhi_data = btc_analyzer.lhi_scores

print("LIQUIDITY HEALTH INDEX SUMMARY")
print("=" * 35)
print(f"Average LHI Score: {lhi_data['lhi_score'].mean():.1f}/100")
print(f"Worst LHI Score: {lhi_data['lhi_score'].min():.1f}/100")
print(f"Best LHI Score: {lhi_data['lhi_score'].max():.1f}/100")

# Distribution of LHI categories
category_counts = lhi_data['lhi_category'].value_counts()
print("\nTime spent in each condition:")
for category, count in category_counts.items():
    pct = (count / len(lhi_data)) * 100
    print(f"  {category}: {pct:.1f}% of time")

# Plot LHI score over time
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# LHI Score
ax1.plot(lhi_data['timestamp'], lhi_data['lhi_score'], color='blue', alpha=0.8, linewidth=1)
ax1.axhline(y=50, color='red', linestyle='--', alpha=0.7, label='Stress Threshold')
ax1.axhline(y=80, color='orange', linestyle='--', alpha=0.7, label='Mild Stress Threshold')
ax1.fill_between(lhi_data['timestamp'], 0, 50, alpha=0.2, color='red', label='Stress Zone')
ax1.fill_between(lhi_data['timestamp'], 50, 80, alpha=0.2, color='orange', label='Caution Zone')
ax1.fill_between(lhi_data['timestamp'], 80, 100, alpha=0.2, color='green', label='Healthy Zone')
ax1.set_ylabel('LHI Score')
ax1.set_title('BTC-USDT Liquidity Health Index Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 100)

# Price for context
ax2.plot(lhi_data['timestamp'], lhi_data['mid_price'], color='black', alpha=0.7, linewidth=1)
ax2.set_ylabel('Price (USDT)')
ax2.set_xlabel('Time')
ax2.set_title('Price Movement (for context)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 4: Identify Stress Periods

This is the practical part - when should traders be extra careful?

We flag periods where:
- Spreads widen beyond 20 bps
- Order book depth drops by 50%+
- Volume spikes 3x+ normal
- LHI score drops below 50

In [ ]:
# Identify stress periods
stress_periods = btc_analyzer.identify_stress_periods()

print(f"IDENTIFIED {len(stress_periods)} STRESS PERIODS")
print("=" * 40)

# Show the worst 5 stress periods
worst_periods = sorted(stress_periods, key=lambda x: x['min_lhi'])[:5]

for i, period in enumerate(worst_periods, 1):
    print(f"{i}. {period['start'].strftime('%m-%d %H:%M')} - {period['end'].strftime('%H:%M')}")
    print(f"   Duration: {period['duration_minutes']} minutes")
    print(f"   Worst LHI: {period['min_lhi']:.1f}/100")
    print(f"   Issues: {period['issues']}")
    print()

# Calculate stress statistics
total_stress_time = sum(p['duration_minutes'] for p in stress_periods)
stress_percentage = (total_stress_time / len(lhi_data)) * 100

print(f"Total time in stress: {total_stress_time} minutes ({stress_percentage:.1f}% of period)")
print(f"Average stress period duration: {total_stress_time/len(stress_periods):.1f} minutes")

## Step 5: Generate Full Analysis Charts

Create the complete liquidity analysis dashboard.

In [ ]:
# Generate the full analysis chart
fig = btc_analyzer.create_charts()
plt.show()

## Step 6: Compare BTC vs ETH

Let's see how the two major pairs compare in terms of liquidity health.

In [ ]:
# Analyze ETH as well
eth_analyzer = LiquidityAnalyzer('ETH-USDT')
eth_analyzer.load_data('data/eth_usdt_data.csv')
eth_analyzer.compute_basic_metrics()
eth_analyzer.compute_lhi_score()
eth_stress_periods = eth_analyzer.identify_stress_periods()

# Compare the two
print("BTC vs ETH LIQUIDITY COMPARISON")
print("=" * 35)

btc_avg_lhi = btc_analyzer.lhi_scores['lhi_score'].mean()
eth_avg_lhi = eth_analyzer.lhi_scores['lhi_score'].mean()

btc_stress_pct = (btc_analyzer.lhi_scores['is_stress'].sum() / len(btc_analyzer.lhi_scores)) * 100
eth_stress_pct = (eth_analyzer.lhi_scores['is_stress'].sum() / len(eth_analyzer.lhi_scores)) * 100

print(f"Average LHI Score:")
print(f"  BTC-USDT: {btc_avg_lhi:.1f}/100")
print(f"  ETH-USDT: {eth_avg_lhi:.1f}/100")
print()
print(f"Time in Stress:")
print(f"  BTC-USDT: {btc_stress_pct:.1f}%")
print(f"  ETH-USDT: {eth_stress_pct:.1f}%")
print()
print(f"Number of Stress Periods:")
print(f"  BTC-USDT: {len(stress_periods)}")
print(f"  ETH-USDT: {len(eth_stress_periods)}")

# Side-by-side LHI comparison
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# BTC LHI
ax1.plot(btc_analyzer.lhi_scores['timestamp'], btc_analyzer.lhi_scores['lhi_score'], 
         color='orange', alpha=0.8, linewidth=1, label='BTC-USDT')
ax1.axhline(y=50, color='red', linestyle='--', alpha=0.5)
ax1.axhline(y=80, color='orange', linestyle='--', alpha=0.5)
ax1.set_ylabel('LHI Score')
ax1.set_title('BTC-USDT Liquidity Health')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 100)

# ETH LHI
ax2.plot(eth_analyzer.lhi_scores['timestamp'], eth_analyzer.lhi_scores['lhi_score'], 
         color='blue', alpha=0.8, linewidth=1, label='ETH-USDT')
ax2.axhline(y=50, color='red', linestyle='--', alpha=0.5)
ax2.axhline(y=80, color='orange', linestyle='--', alpha=0.5)
ax2.set_ylabel('LHI Score')
ax2.set_xlabel('Time')
ax2.set_title('ETH-USDT Liquidity Health')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

## Key Takeaways for Trade Ops

**What This System Does**:
- Combines spread, depth, volume, and volatility into a single 0-100 health score
- Flags periods when execution conditions are poor
- Provides specific reasons why liquidity is stressed

**How to Use It**:
- LHI > 80: Normal trading
- LHI 50-80: Be cautious, consider splitting large orders
- LHI < 50: Reduce size, use limit orders, avoid market orders

**Limitations** (being honest about what this doesn't do):
- Based on top-of-book data only (no deep order book analysis)
- Thresholds are manually set (not dynamically optimized)
- Doesn't account for cross-exchange arbitrage opportunities
- Volume spikes might be legitimate (not always bad)

**Next Steps**:
- Monitor system performance against actual execution quality
- Adjust thresholds based on real trading experience
- Consider adding more sophisticated order book analysis
- Integrate with trading systems for real-time alerts